In [7]:
from openai import OpenAI
import json
import os
import csv

In [8]:
client=OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [9]:
def dataCleanup(fileLocation):
    questions=[]
    with open(fileLocation,mode="r",encoding='utf-8') as csv_file:
        csv_reader=csv.DictReader(csv_file)
        for row in csv_reader:
            questions.append(row["questions"])
    return questions

questions=dataCleanup("./resources/questions_with_topic.csv")
questions

['Describe the concept of *thermal runaway* in IGBT devices.',
 'What is voltage-controlled oscillator?.',
 'How does a piezoelectric sensor generate an electrical signal?.',
 'Describe the gate drive requirements for an IGBT.',
 'What is a flip-flop and how is it used in digital circuits?',
 'Discuss the merits of using a bridge rectifier in voltmeter circuits for accurate AC voltage measurement.',
 'What is the significance of the **holding current** in an SCR?',
 'What is Tuned Oscillator Circuit?.',
 'Explain the principle of *soft switching* in IGBT applications.',
 'Describe the regions for Transistor?.',
 'Describe the basic working principle of a light dimmer using a triac.',
 "What is the significance of Moore's Law in the context of integrated circuits?",
 'What is the significance of *switching losses* in IGBT operation?',
 'What are the advantages of using a CMOS technology over NMOS or PMOS alone?.',
 'Describe three terminals of MOSFET?.',
 'What are the applications of p

In [10]:
len(questions)

599

In [11]:
answer_prompt="""
The following is a piece of dialogue between a student and a teacher.\n
Answer the latest question asked by an undergraduate student as if you were a university professor of Electrical and Electronic Engineering of a university.\n
First, tell which specific terms, theories, and information need to be learned prior to the query in a sentence. Avoid explaining them. 
If the question requires an explanation, provide an explanation divided into short and simple steps.\n
If the question can be answered in short, keep it short.\n
Only answer what is asked. Stay inside the topic. Provide correct, easily understandable, and concise answers.\n
You will be penalized 1999 dollars for irrelevant, incorrect, verbose, or hard-to-understand answers.\n
Avoid addressing the student.\n\n
"""

In [12]:
question_prompt="""
The following is a piece of dialogue between a student and a teacher.\n
Ask a question based on the context and content of the dialogue as if you where an undergraduate electrical and electronic engineering student asking your professor.\n
The question should be insightful, short, and diverse.\n
You will be penalized 1999 dollars if you ask question that is same or similar to any previous question.\n
You will be penalized 2999 dollars if you provide feedback instead of a question.\n
Avoid any kind of addressing.\n\n
"""

In [14]:
conversationLevel=4
questions=["Describe the concept of *thermal runaway* in IGBT devices"]
for idx,question in enumerate(questions):
    # if idx<=300:
    #     continue
    converationArray=[]

    conversationHistoryAnswering=[
        {
        "role": "system",
        "content": answer_prompt, 
        },
        {
        "role": "user",
        "content": question,
        }
    ]
    
    # For generating questions
    conversationHistoryQuestioning=[
        {
        "role": "system",
        "content": question_prompt,
        },
        {
        "role": "user",
        "content": question,
        }
    ]

    
    currentQuestion=question
    for p in range(conversationLevel):

        #TEACHER PART........................................................................
        teacher = client.chat.completions.create(
            messages=conversationHistoryAnswering,
            model="gpt-4o",
        )
        teacherAnswer = teacher.choices[0].message.content


        #append the teacher answer.......................
        conversationHistoryAnswering.append({
            "role": "assistant",
            "content": teacherAnswer,
        })
        conversationHistoryQuestioning.append({
            "role": "assistant",
            "content": teacherAnswer,
        })


        #STUDENT PART........................................................................
        student = client.chat.completions.create(
            messages=conversationHistoryQuestioning,
            model="gpt-4o",
        )
        studentQuestion = student.choices[0].message.content

        conversationHistoryAnswering.append({
            "role": "user",
            "content": studentQuestion,
        })
        conversationHistoryQuestioning.append({
            "role": "user",
            "content": studentQuestion,
        })
        print("LEVEL ",p,"\n======================================================================")
        print("Student : ",currentQuestion)
        print("Teacher : ",teacherAnswer)
        currentQuestion=studentQuestion
        
        converationArray.append(teacherAnswer)
        converationArray.append(studentQuestion)
    data = {
        "question1": question,
        "answer1": converationArray[0],
        "question2": converationArray[1],
        "answer2": converationArray[2],
        "question3": converationArray[3],
        "answer3": converationArray[4],
        "question4": converationArray[5],
        "answer4": converationArray[6],
    }
    filename = "output.csv"

    # Define the fieldnames for the CSV file
    fieldnames = ["question1", "answer1", "question2","answer2","question3","answer3","question4","answer4"]

    # Check if the file exists
    file_exists = os.path.isfile(filename)

    # Write data to CSV file
    with open(filename, 'a', newline='', encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
        # Write header if file doesn't exist
        if not file_exists:
            writer.writeheader()
    
        # Write data row
        writer.writerow(data)

LEVEL  0 
Student :  Describe the concept of *thermal runaway* in IGBT devices
Teacher :  Before addressing thermal runaway in IGBT devices, you should understand concepts such as Insulated Gate Bipolar Transistor (IGBT) structure and operation, power dissipation, junction temperature, and thermal management in electronic devices.

1. **Definition**: Thermal runaway in IGBT devices occurs when an increase in temperature leads to higher power loss, which further increases the temperature in a self-reinforcing cycle, potentially leading to device failure.

2. **Key Factor**: An increase in temperature reduces the on-state voltage drop of an IGBT, increasing current flow and, consequently, power dissipation.

3. **Cycle**: This increased power dissipation raises the junction temperature, further reducing the on-state voltage drop, leading to a continuous cycle of increasing temperature and power loss.

4. **Outcome**: If unchecked, thermal runaway can cause the IGBT to overheat and fail d